In [197]:
import sys
import numpy as np
import matplotlib.pyplot as plt

sys.path.append('./')
from planar_ising.planar_ising.inference_and_sampling import InferenceAndSampling
from planar_ising.planar_ising.planar_ising_model_generator import PlanarIsingModelGenerator
from planar_ising.planar_graph.planar_graph_edges import PlanarGraphEdges
from planar_ising.planar_graph.planar_graph_generator import PlanarGraphGenerator

from triangular_lattice_v4 import *

%load_ext autoreload
%autoreload 2

def partition_function_method(sets,G,double_measurements):
    q=2
    beta = np.log((q**2+1)/q)

    jmatrix,E0 = model_couplings(sets,G,double_measurements,tp = 'num') 
    jmatrix_planar = -beta*jmatrix
    ising_model = PlanarIsingModelGenerator.generate_triang_ising_model(database, jmatrix_planar)
    ising_inference = InferenceAndSampling(ising_model)
    ising_inference.prepare()
    log_partition_function = ising_inference.compute_logpf()
    Snum  = -log_partition_function+beta*E0-np.log(0.5)

    jmatrix,E0 = model_couplings(sets,G,double_measurements,tp = 'denom') 
    jmatrix_planar = -beta*jmatrix
    ising_model = PlanarIsingModelGenerator.generate_triang_ising_model(database, jmatrix_planar)
    ising_inference = InferenceAndSampling(ising_model)
    ising_inference.prepare()
    log_partition_function = ising_inference.compute_logpf()
    Sden  = -log_partition_function+beta*E0-np.log(0.5)
    return Snum-Sden


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [199]:
Lph,T = 16,16
L = int(Lph/2)+(int(Lph/2)%2)
p = 1-(int(Lph/2)%2)

# measurement probability
prob=0.25
G=8

#=================================================================================

basisx,basisy,sets,measured,sole_measurements,double_measurements = trnglr_lattice(L,Lph,T,p,prob=prob)
adjmatrix = allOR(sets)
database = list_edges(adjmatrix,basisx,basisy)  

##=================================================================================

S1 = cellular_automaton_method(Lph,T,measured,smp_std=100000)
S2 = partition_function_method(sets,G,double_measurements)

print('CA method:',S1)
print('PF method:',S2)

CA method: 0.35435784258325587
PF method: 0.35763774026486317
